# 1. Defining the Question


a) Specifying the Question




b) Defining the Metric for Success


c) Understanding the context


d) Recording the Experimental Design


e) Data Relevance

In [0]:
#Import libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split

# Import LinearRegression method from sklearn linear_model library
from sklearn.linear_model import LinearRegression

# 2. Reading the Data

In [0]:
# Reading in the datasets using the Pandas
# 
fifa = pd.read_csv("fifa_ranking.csv", parse_dates=['rank_date'])
fifa

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07


In [0]:
# function to extract the year from the rank_date column

def extract_year(fifa, column):
    fifa["year"] = fifa[column].apply(lambda x: x.year)
    
extract_year(fifa, 'rank_date')
fifa.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date,year
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08,1993
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,1993


In [0]:
# Dropping the unneccesary columns in the fifa dataset
fifa.drop(fifa.columns.difference(['rank','country_full' , 'year']), 1, inplace=True)
fifa

,rank,country_full,year
0,1,Germany,1993
1,2,Italy,1993
2,3,Switzerland,1993
3,4,Sweden,1993
4,5,Argentina,1993
...,...,...,...
57788,206,Anguilla,2018
57789,206,Bahamas,2018
57790,206,Eritrea,2018
57791,206,Somalia,2018


In [0]:
results = pd.read_csv("results.csv", parse_dates=['date'])
results
# to get the dates to align with the fifa dataset we will subset 

results1 = results[results['date']>= '1993-08-08']
results1

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
17915,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False
17916,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False
17917,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False
17918,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False
17919,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False
...,...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True


In [0]:
# function to extract the year from the date column

def extract_year(results1, column):
    results1["year"] = results1[column].apply(lambda x: x.year)
    
extract_year(results1, 'date')
results1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
17915,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,1993
17916,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,1993
17917,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,1993
17918,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,1993
17919,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,1993


In [0]:
# Dropping the unnecessary columns in results1 dataset
# we first subset to get the games that did not have a neutral field
results2 = results1[results1['neutral']== False]
results2

results2.drop(['date','city','neutral'],1, inplace=True)
results2


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,home_team,away_team,home_score,away_score,tournament,country,year
17915,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,1993
17916,Brazil,Mexico,1,1,Friendly,Brazil,1993
17917,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Ecuador,1993
17918,Guinea,Sierra Leone,1,0,Friendly,Guinea,1993
17919,Paraguay,Argentina,1,3,FIFA World Cup qualification,Paraguay,1993
...,...,...,...,...,...,...,...
40803,Samoa,Papua New Guinea,0,6,Pacific Games,Samoa,2019
40817,Samoa,Tonga,2,0,Pacific Games,Samoa,2019
40821,India,North Korea,2,5,Intercontinental Cup,India,2019
40829,India,Syria,1,1,Intercontinental Cup,India,2019


In [42]:
df3 = pd.merge(fifa, results2, how='left', left_on=['country_full', 'year'], right_on=['home_team', 'year'])
df3
def away_team_ranker(df, column1, column2, column3):
  for i in len(column1):
    for j in len(column2):
      if list(column1[df[i]]) == list(column2[df[j]]):
        list(df['away team rank'])= list(column3[df[j]])
        return(df['away team rank'])

away_team_ranker(df3, 'away_team', 'country_full', 	'rank')
df3.head()
   

SyntaxError: ignored

In [37]:
 if df3['home_team'] == df3[':

no


# 3. Checking the Data

In [0]:
# getting information on the fifa dataset
#
fifa.info()

In [0]:
# getting information on the results dataset
#
results_subset.info()